# Pipelines 

Working without Pipelines

In [32]:
import pandas as pd 
import numpy as np

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import   DecisionTreeClassifier

In [34]:
df =pd.read_csv('dataset\\train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [35]:
df.drop(columns = ['PassengerId','Name','Ticket','Cabin'],inplace =True)


In [36]:
df.sample(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
805,0,3,male,31.0,0,0,7.775,S
543,1,2,male,32.0,1,0,26.000,S
717,1,2,female,27.0,0,0,10.500,S
188,0,3,male,40.0,1,1,15.500,Q
733,0,2,male,23.0,0,0,13.000,S


train_test_split   data

In [37]:
# Step 1--> train_test_split 
X_train,X_test,Y_train,Y_test = train_test_split(df.drop(columns =['Survived']) ,df['Survived'],test_size =.2  ,random_state = 42)


In [38]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5,S
733,2,male,23.0,0,0,13.0,S


In [39]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

  Applying SimpleImputer  for missing values 

In [40]:
#step --->Imputation
si_age =SimpleImputer()  # filling with mean

si_embarked =SimpleImputer(strategy='most_frequent')  # filling with most frequent


X_train_age =si_age.fit_transform(X_train[['Age']])
X_train_embarked =si_embarked.fit_transform(X_train[['Embarked']])


X_test_age = si_age.transform(X_test[['Age']])
X_test_embarked =si_embarked.transform(X_test[['Embarked']])



ONEHOT Encoding on Non order data(Nominal data)

In [75]:
#Step 3----> Onehot encoding on Sex and Embarked
ohe_sex =OneHotEncoder(sparse_output=False,handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

X_train_sex =ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked =ohe_embarked.fit_transform(X_train_embarked)


X_test_sex = ohe_sex.transform(X_test[['Sex']])
X_test_embarked =ohe_embarked.transform(X_test_embarked) 

droping the od column from X_train of Sex, Embarked and Age So that the transform column can be added

In [76]:
X_train_rem = X_train.drop(columns =['Sex','Age','Embarked'])
X_test_rem = X_test.drop(columns = ['Sex','Age','Embarked'])

In [77]:
print(X_test_rem.shape)
print(X_test_age.shape)
print(X_test_sex.shape)
print(X_test_embarked.shape)


(179, 4)
(179, 1)
(179, 2)
(179, 24)


In [78]:
X_train_transformed = np.concatenate((X_train_rem,X_train_age,X_train_embarked,X_train_sex),axis =1)


In [79]:
X_test_transformed =np.concatenate((X_test_rem,X_test_age,X_test_embarked,X_test_sex),axis =1)

Decision Tree Classifier Implementation


In [80]:
clf =DecisionTreeClassifier()
clf.fit(X_train_transformed,Y_train)

DecisionTreeClassifier()

In [81]:
Y_predict =clf.predict(X_test_transformed)

In [82]:
print(Y_predict)

[0 1 0 1 1 0 1 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0 1
 1 1 0 0 1 0 0 0 0 0 0 1 0 1 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0 0 0 1 1 1 1 1
 0 0 1 1 1 1 0 1 1 0 1 0 1 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 1 0 0 0 1
 0 1 1 0 0 0 0 1 0 0 1 1 1 0 0 1 1 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 1 0 1
 1 0 0 1 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 1 1]


In [84]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,Y_predict,)

0.770949720670391

Exporting the data so that people can check after entering new data that the person will be survived or not

Deploying Model on Website

In [85]:
import pickle

In [87]:
import os

# Create 'models' directory if it doesn't exist
os.makedirs('models', exist_ok=True)

In [88]:
pickle.dump(ohe_sex,open('models/ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked,open('models/ohe_embarked.pkl','wb'))
pickle.dump(clf,open('models/clf.pkl','wb'))

 without pipeline i have to go through all the process again for a new input 

In [90]:
# few things

# With Pipelines

In [91]:
from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import   DecisionTreeClassifier

In [92]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest,chi2

In [93]:
from sklearn.pipeline import Pipeline,make_pipeline

In [94]:
df =pd.read_csv('dataset\\train.csv')

Pipeline Workflow

Let's Plan ---> 1. Imputation

                2. OnehotEncoding

                3. Scaling

                4. Feature Selection only (5 column)
                
                5. Model Training (DecisionTreeClassifer)

In [95]:
df.drop(columns = ['PassengerId','Name','Ticket','Cabin'],inplace =True)


In [96]:
# train_test_split
A_train, A_test ,B_train,B_test =train_test_split(df.drop(columns =['Survived']),df['Survived'],test_size =.2,random_state=42)

In [111]:
# Imputation  Transformer 
trf1 = ColumnTransformer(transformers=[('impute_age',SimpleImputer(),[2]),
                         ('impute_embarked',SimpleImputer(strategy='most_frequent') ,[6])], remainder='passthrough')
                         

In [112]:
# one hot Encoding
trf2 = ColumnTransformer(transformers=[('ohe_sex_embarked',OneHotEncoder(sparse_output= False,handle_unknown='ignore'),[1,6])],remainder='passthrough')

In [113]:
#Scaling
trf3 =ColumnTransformer(transformers=[('scale',MinMaxScaler(),slice(0,10))]) # slice confirm that its is apply from 0 to 10 columns

In [101]:
# Feature Selection
trf4 =SelectKBest(score_func=chi2,k=8) #k=8 represnts top 8 columns

In [114]:
trf5 = DecisionTreeClassifier()

Create Pipeline

In [115]:
pipe =Pipeline([('trf1',trf1),
                ('trf2',trf2),
                ('trf3',trf3),
                ('trf4',trf4),
                ('trf5',trf5)])

# Pipeline vs make_pipeline

Pipelines requires naming of steps ,make_pipeline does not (Same appilies to Column transformer vs make_column_tranformer)

In [116]:
#alternative Syntax 
# make_pipeline
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [117]:
# train
pipe.fit(A_train,B_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000002A3F0879080>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

# Explore the Pipeline

In [125]:
pipe.named_steps # display the steps pf pipeline

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x000002A3F0879080>),
 'decisiontreeclassifier': DecisionTreeClassifier()}

Explore by yourself 

In [130]:
#pipe.named_steps['trf1'].transformers_

In [131]:
# Display Pipeline
from sklearn import set_config
set_config(display='diagram')

In [132]:
# predict
B_predict = pipe.predict(A_test)

In [133]:
print(B_predict)

[1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0
 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1
 0 0 0 1 0 0 0 0 1 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0
 0 1 0 0 0 1 1 0 0 0 1 0 0 0 1 0 1 1 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0]


In [135]:
from sklearn.metrics import accuracy_score
accuracy_score(B_test,B_predict)

0.6256983240223464

# Cross Validation Using Pipeline

In [136]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,A_train,B_train,cv=5,scoring='accuracy').mean()

0.6391214419383433

# GridSearch using Pipeline

In [ ]:
# grid searchcv
params = {
    'trf5_max_depth':[1,2,3,4,5]
}